In [10]:
from bs4 import BeautifulSoup
import pandas as pd 
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [11]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode (no UI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [12]:
#stockist-result stockist-list-result

In [13]:
# Load the page
url1 = 'https://www.superga.com/pages/store-locator' # Construct the full URL
driver.get(url1)

# Wait for JavaScript to render the page (Adjust time based on page load)
time.sleep(5)

# Now that the page is rendered, extract the page source
html_content = driver.page_source

# Parse the page source with BeautifulSoup
store_soup = BeautifulSoup(html_content, 'html.parser')

print(store_soup)

<html class="js" dir="ltr" lang="en" style="overflow: hidden;"><head><style>.LGLeeN-keyboard-shortcuts-view{display:-webkit-box;display:-webkit-flex;display:-moz-box;display:-ms-flexbox;display:flex}.LGLeeN-keyboard-shortcuts-view table,.LGLeeN-keyboard-shortcuts-view tbody,.LGLeeN-keyboard-shortcuts-view td,.LGLeeN-keyboard-shortcuts-view tr{background:inherit;border:none;margin:0;padding:0}.LGLeeN-keyboard-shortcuts-view table{display:table}.LGLeeN-keyboard-shortcuts-view tr{display:table-row}.LGLeeN-keyboard-shortcuts-view td{-moz-box-sizing:border-box;box-sizing:border-box;display:table-cell;color:#000;padding:6px;vertical-align:middle;white-space:nowrap}.LGLeeN-keyboard-shortcuts-view td:first-child{text-align:end}.LGLeeN-keyboard-shortcuts-view td kbd{background-color:#e8eaed;border-radius:2px;border:none;-moz-box-sizing:border-box;box-sizing:border-box;color:inherit;display:inline-block;font-family:Google Sans Text,Roboto,Arial,sans-serif;line-height:16px;margin:0 2px;min-height

In [14]:
# Ensure store_soup is a BeautifulSoup object and contains the HTML
stores = store_soup.find_all('li', class_='stockist-result stockist-list-result')

# Check the result by printing each store element's prettified HTML for clarity
for store in stores:
    print(store.prettify())  # Print the store HTML in a readable format

# Alternatively, just print the length of stores to confirm the number of matches
print(f"Number of stores found: {len(stores)}")

<li class="stockist-result stockist-list-result" data-stockist-filters="flt_gpy2v923 flt_zp4r858p" data-stockist-location="loc_4n2kyrn5">
 <div class="stockist-result-distance">
  <span aria-hidden="true" class="stockist-icon stockist-icon-marker stockist-feature-color">
  </span>
 </div>
 <div class="stockist-result-name stockist-feature-color">
  Superga 17 Torino
 </div>
 <div class="stockist-result-address">
  <div class="stockist-result-addr-1">
   Via Lagrange 27
  </div>
  <div class="stockist-result-addr-locality">
   Torino, TO  10123
  </div>
  <div class="stockist-result-addr-country">
   Italy
  </div>
 </div>
 <div class="stockist-result-filters">
  <div class="stockist-result-filter stockist-result-filter-gpy2v923">
   Store
  </div>
  <div class="stockist-result-filter stockist-result-filter-zp4r858p">
   Punti reso
  </div>
 </div>
 <div class="stockist-result-details">
  <div class="stockist-result-phone">
   <a class="stockist-feature-color" href="tel:0114546070">
   

In [15]:
rows = []
for store in stores:
    # Extract the store name
    store_name = store.find('div', class_='stockist-result-name').text.strip()

    # Extract the address
    address = store.find('div', class_='stockist-result-addr-1').text.strip()

    # Extract locality (city, state, postal code)
    locality = store.find('div', class_='stockist-result-addr-locality').text.strip()

    # Split locality into city and state/postal code
    city, state_postal = locality.split(',', 1)
    city = city.strip()
    state_postal = state_postal.strip()

    # Split state/postal into state and postal code, make sure postal code is treated as a string
    state, postal_code = state_postal.split(' ', 1)
    postal_code = postal_code.strip()  # Ensure it's kept as a string with any leading zeros

    # Extract email, check if it exists
    email_div = store.find('div', class_='stockist-result-email')
    email = email_div.a.text.strip() if email_div else "No email"

    # Extract VAT number (ensure it's treated as a string)
    vat_element = store.find('a', class_='stockist-feature-color')
    vat = vat_element.text.strip() if vat_element else "No VAT"
    
    # Make sure VAT is treated as a string
    vat = str(vat)

    # Append the data to rows
    rows.append({
        'Store_name': store_name,
        'address': address,
        'locality': locality,
        'city': city,
        'state_postal': state_postal,
        'state': state,
        'postal_code': postal_code,  # Postal code remains a string
        'email': email,
        'vat': vat  # VAT is kept as string
    })

In [16]:
df = pd.DataFrame(rows)

In [17]:
print(df)

                    Store_name                            address  \
0            Superga 17 Torino                    Via Lagrange 27   
1            Superga 60 Torino                Corso De Gasperi 29   
2              Superga 30 Roma             Via Di Campo Marzio 33   
3              Superga 31 Roma                  Via Della Vite 86   
4           Superga 200 Milano              Via Ragazzi Del '99 2   
5            Superga 43 Milano                   Corso 22 Marzo 9   
6            Superga 1 Caselle             Strada San Maurizio 12   
7          Superga 101 Palermo   Via Giuseppe Lanza Di Scalea Snc   
8          Superga 109 Trapani  Via Giovanbattista Fardella 178/A   
9          Superga 117 Salerno           Via Dei Principati 58/60   
10           Superga 120 Cuneo                        Via Roma 36   
11  Superga 122 Riva del Garda                Via Armando Diaz 20   
12         Superga 136 Catania           Via Cristoforo Colombo 1   
13         Superga 145 Catania    

In [18]:
df.to_csv('storesuperga.csv', index=False)